In [ ]:
from google.cloud import bigquery
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

json = os.getenv("JSON_SUPER_SECRETO")

project = os.getenv("BQ_PROJECT")
dataset = os.getenv("BQ_DATASET")

#tabelas
tabela_tratado = os.getenv("BQ_TABELA_TRATADO")

client = bigquery.Client.from_service_account_json(json)

In [ ]:
#Query - Full Tratado
query_items = f"""
SELECT
*
FROM `{project}.{dataset}.{tabela_tratado}`
ORDER BY data;
"""

query_full = client.query(query_items)
rows_full = query_full.result()
df_full = pd.DataFrame([dict(rows_full) for rows_full in rows_full])

In [43]:
#pegando o mes e ano do df p/ indice
df_full["data"] = pd.to_datetime(df_full["data"])
df_full["mes_ano"] = df_full["data"].dt.to_period("M")

print(df_full["mes_ano"].dtype)

period[M]


In [39]:
#criar o indice de saude GERAL e MENSAL (funcao p/ evitar repetição e possíveis erros)
def criar_indice_saude(df, group_by):
    def nota(x):
        return (x - x.min()) / (x.max() - x.min()) * 10

    #group by das métricas
    df_agregar = df.groupby(group_by).agg(
        faturamento=("total_value", "sum"),
        total_itens=("qtd_itens", "sum"),
        total_descontos=("valor_descontos", "sum"),
        torque=("net_torque", "mean"),
        torque_delivery=("net_torque_delivery", "mean"),
        torque_em_loja=("net_torque_in_store", "mean")
    ).reset_index()

    #dando uma nota para cada métrica
    df_agregar["faturamento_nota"] = nota(df_agregar["faturamento"])
    df_agregar["torque_nota"] = nota(df_agregar["torque"])
    df_agregar["torque_delivery_nota"] = nota(df_agregar["torque_delivery"])
    df_agregar["torque_em_loja_nota"] = nota(df_agregar["torque_em_loja"])
    df_agregar["total_itens_nota"] = nota(df_agregar["total_itens"])
    df_agregar["descontos_nota"] = nota(df_agregar["total_descontos"])

    #criando o indice de saude
    df_agregar["indice_saude"] = (
        0.3 * df_agregar["faturamento_nota"] +
        0.2 * df_agregar["torque_nota"] +
        0.2 * df_agregar["torque_delivery_nota"] +
        0.2 * df_agregar["torque_em_loja_nota"] +
        0.1 * df_agregar["total_itens_nota"] +
        0.1 * df_agregar["descontos_nota"]
    )

    return df_agregar

In [48]:
#criar os indices geral e mensal
df_lojas_indice_saude_geral = criar_indice_saude(df_full, ["shop_id"])
df_lojas_indice_saude_mensal = criar_indice_saude(df_full, ["shop_id", "mes_ano"])
df_lojas_indice_saude_mensal['mes_ano'] = df_lojas_indice_saude_mensal['mes_ano'].dt.date

print(df_lojas_indice_saude_geral.head(10))
print(df_lojas_indice_saude_mensal.head(10))

            shop_id  faturamento  total_itens  total_descontos     torque  \
0   Adidas Shopping   1931131.05     215343.0         22333.86  52.369669   
1  Av. Nova Balança   1424611.57     115640.0         22704.88  56.211043   
2        Praça Ekin   1520361.61     147853.0         24576.97  48.328948   
3  R. Bens Perdidos   1827588.29     141087.0         38150.95  51.572738   

   torque_delivery  torque_em_loja  faturamento_nota  torque_nota  \
0        44.172401       50.447593         10.000000     5.126456   
1        39.492213       55.703733          0.000000    10.000000   
2        47.720985       47.181092          1.890353     0.000000   
3        14.764466       51.494213          7.955799     4.115391   

   torque_delivery_nota  torque_em_loja_nota  total_itens_nota  \
0              8.923253             3.832734         10.000000   
1              7.503143            10.000000          0.000000   
2             10.000000             0.000000          3.230896   
3   

In [49]:
#exportar
#df_lojas_indice_saude_geral.to_csv("indice_saude_lojas_geral.csv", index=False)
df_lojas_indice_saude_mensal.to_csv("indice_saude_lojas_mensal.csv", index=False)